In [4]:
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from PIL import Image

In [5]:
train_path = 'DATASET/TRAIN'
test_path = 'DATASET/TEST'
IMG_BREDTH = 30
IMG_HEIGHT = 60
num_classes = 2

In [6]:
train_batch = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=45, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=False).flow_from_directory(train_path, 
                                                                          target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                                          classes=['O', 'R'], 
                                                                          batch_size=100)

test_batch = ImageDataGenerator().flow_from_directory(test_path, 
                                                      target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                      classes=['O', 'R'], 
                                                      batch_size=100)


Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [7]:
def cnn_model():
    
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(IMG_HEIGHT,IMG_BREDTH,3)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
              
    model.summary()
              
    return model

In [8]:
def use_model(path):
    
    # model = load_model('best_waste_classifier.h5')
    pic = plt.imread(path)
    pic = cv2.resize(pic, (IMG_BREDTH, IMG_HEIGHT))
    pic = np.expand_dims(pic, axis=0)
    classes = model.predict_classes(pic)
    

    return classes


In [11]:
model = cnn_model()
callbacks =[ ModelCheckpoint('best_waste_classifier.h5', 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min'),
                             ReduceLROnPlateau(monitor='val_loss', mode= 'min',
                             factor= 0.75, verbose= True, patience=2)]
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])

model = model.fit_generator(train_batch,  
                            validation_data=test_batch,  
                            epochs=20, 
                            verbose=1,
                            callbacks= callbacks)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 60, 30, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 58, 28, 32)        9248      
                                                                 
 conv2d_8 (Conv2D)           (None, 56, 26, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 28, 13, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 26, 11, 64)        18496     
                                                      

C:\Users\kunwa\AppData\Local\Temp\ipykernel_34960\528939447.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model = model.fit_generator(train_batch,


Epoch 1/20
226/226 [==============================] - ETA: 0s - loss: 0.8749 - accuracy: 0.7293
Epoch 1: val_accuracy improved from inf to 0.78750, saving model to best_waste_classifier.h5
226/226 [==============================] - 253s 1s/step - loss: 0.8749 - accuracy: 0.7293 - val_loss: 0.4745 - val_accuracy: 0.7875 - lr: 0.0010
Epoch 2/20
226/226 [==============================] - ETA: 0s - loss: 0.4708 - accuracy: 0.7925
Epoch 2: val_accuracy did not improve from 0.78750
226/226 [==============================] - 231s 1s/step - loss: 0.4708 - accuracy: 0.7925 - val_loss: 0.3492 - val_accuracy: 0.8691 - lr: 0.0010
Epoch 3/20
226/226 [==============================] - ETA: 0s - loss: 0.4526 - accuracy: 0.7991
Epoch 3: val_accuracy did not improve from 0.78750
226/226 [==============================] - 261s 1s/step - loss: 0.4526 - accuracy: 0.7991 - val_loss: 0.3820 - val_accuracy: 0.8321 - lr: 0.0010
Epoch 4/20
226/226 [==============================] - ETA: 0s - loss: 0.4433 - acc